## Question 4: Segmentation

In this question, we perform segmentation on a given image in order to
extract segment that belong to pigeons. As always, functions are written in
`q4_funcs.py` and main code that uses these functions is in
`q4.py`

### q4_funcs

Different methods of segmentation is implemented in this file, but since we have used Felzenszwalb method
in the main problem, we will just explain that part :

Function `get_feature_vectors` gets an image as input and
outputs feature vectors used for checking similarities. In this problem, we just give the original BGR
image as feature vector :

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from skimage.segmentation import mark_boundaries
from skimage.segmentation import slic, felzenszwalb

def get_feature_vectors(img):
    # lab_vectors = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lab_vectors = img.copy()
    return lab_vectors

Function `segment_felzenszwalb` takes an image and other parameters used in Felzenszwalb and returns
segmented result along with labels

In [ ]:
def segment_felzenszwalb(img, scale, sigma, min_size):
    labels = felzenszwalb(img, scale, sigma, min_size)
    result = mark_boundaries(img, labels, color=(0, 0, 0))
    return result, labels


Next function takes a super pixel and a sample, and check how much they are similar.
For checking, we take the feature vectors of the pixels in the given super pixel, calculate their average, and
compare it with the given sample:

In [ ]:
def check_similarity(lab_vectors, super_pixel, sample, threshold):
    super_pixel_value = np.average(np.average(lab_vectors[super_pixel], axis=0), axis=0)
    diff = np.sqrt(np.sum(np.square(sample - super_pixel_value)))
    print(diff)
    if diff < threshold:
        return True
    return False


Finally, function `find_matches` takes an image, feature vectors, labels and sample as
input and outputs an array matches, containing the label of the matched clusters and
a result, which is the image with similar super pixels being black :

In [ ]:
def find_matches(img, lab_vectors, labels, sample):
    result = img.copy()
    matches = []
    m, n = np.min(labels), np.max(labels)
    print(m, n)
    for i in range(m, n + 1):
        super_pixel_i = np.where(labels == i)
        if check_similarity(lab_vectors, super_pixel_i, sample, 50):
            matches.append(i)
            result[super_pixel_i] = 0
    return matches, result

### q1.py
In main code, we have sample vector computed by hand ( by taking some samples from
a pigeon in the image and computing their average), and we take only a part of image that contains all the
pigeons, and use above functions to find pigeons :

In [ ]:
import q4_funcs
import matplotlib.pyplot as plt
import numpy as np
import cv2
from skimage.segmentation import mark_boundaries
from skimage.segmentation import slic, felzenszwalb

input_path = "inputs/birds.jpg"
output_path = "outputs/res10.jpg"
img = cv2.imread(input_path)

result, labels = q4_funcs.segment_felzenszwalb(img, 1000, 0.6, 100)

lab_vectors = q4_funcs.get_feature_vectors(img)

# sample_img = img[1910:2020, 2200:2275, :]
# result1, labels1 = q4_funcs.segment_felzenszwalb(sample_img, 800, 0.6, 100)
# plt.imshow(result1)
# plt.show()

sample = np.asarray([97, 103, 109])
# sample = q4_funcs.calculate_sample_value(lab_vectors, 1910, 2200, 110, 75)
matches, result = q4_funcs.find_matches(img[1500:2350, 220:, :], lab_vectors[1500:2350, 220:, :],
                                        labels[1500:2350, 220:],
                                        sample)
# q4_funcs.draw_rectangles(result, labels, matches, (110, 75), (1500, 220))
plt.imshow(result)
plt.show()
img_copy = img[1500:2350, 220:, :].copy()
mask = np.where(result == 0)
img_copy[result != 0] = 0

plt.imshow(img_copy)
plt.show()
final_result = np.zeros(img.shape, dtype='uint8')
final_result[1500:2350, 220:, :] = img_copy
cv2.imwrite(output_path, final_result)